In [1]:
from itertools import count
from collections import defaultdict
from scipy.sparse import csr
import numpy as np
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
import tensorflow as tf
from tqdm import tqdm_notebook as tqdm

from scipy.sparse import lil_matrix
from scipy.sparse import csr

import csv

FM用于电影评分预测

数据集格式

userid | itemid | rating | timestamp


数据结构

- a training sparse matrix: 90,570 lines and 2,625 columns (943 one-hot encoded features for the user id, plus 1682 one-hot encoded features for the movie id),
- a training label array: 90,570 ratings,
- a test sparse matrix: 9,430 lines and 2,625 columns,
- a test label array: 9,430 ratings.


In [2]:
!shuf data/ua.base -o data/ua.base.shuffled
!head -10 data/ua.base.shuffled

525	405	4	881086693
286	290	3	876522072
919	1012	4	875289611
418	899	5	891282706
316	1039	5	880854500
523	257	5	883700187
854	237	3	882812406
327	715	4	887819860
308	420	4	887740216
469	238	4	879525237


In [3]:
nbUsers=943
nbMovies=1682
nbFeatures=nbUsers+nbMovies

nbRatingsTrain=90570
nbRatingsTest=9430

# 数据集
def load_data(filename, lines, columns):
    # Features are one-hot encoded in a sparse matrix
    # userid, itemid one-hot vector 2625 dim (943+1682) two values set 1
    X = lil_matrix((lines, columns)).astype('float32')
    # Labels are stored in a vector
    Y = []
    line=0
    with open(filename,'r') as f:
        samples=csv.reader(f,delimiter='\t')
        for userId,movieId,rating,timestamp in samples:
            X[line,int(userId)-1] = 1
            X[line,int(nbUsers)+int(movieId)-1] = 1
            if int(rating) >= 3:
                Y.append(1)
            else:
                Y.append(0)
            line=line+1
            
    Y=np.array(Y).astype('float32')
    return X,Y

In [12]:
x_train, y_train = load_data('data/ua.base.shuffled', nbRatingsTrain, nbFeatures)
x_test, y_test = load_data('data/ua.test',nbRatingsTest,nbFeatures)

In [13]:
type(x_train)

scipy.sparse.lil.lil_matrix

In [14]:
print(x_train.shape)
print(y_train.shape)
assert x_train.shape == (nbRatingsTrain, nbFeatures)
assert y_train.shape == (nbRatingsTrain, )
zero_labels = np.count_nonzero(y_train)
print("Training labels: %d zeros, %d ones" % (zero_labels, nbRatingsTrain-zero_labels))

print(x_test.shape)
print(y_test.shape)
assert x_test.shape  == (nbRatingsTest, nbFeatures)
assert y_test.shape  == (nbRatingsTest, )
zero_labels = np.count_nonzero(y_test)
print("Test labels: %d zeros, %d ones" % (zero_labels, nbRatingsTest-zero_labels))

(90570, 2625)
(90570,)
Training labels: 74627 zeros, 15943 ones
(9430, 2625)
(9430,)
Test labels: 7893 zeros, 1537 ones


In [15]:
x_train = x_train.tocsr()
x_test = x_test.tocsr()

In [9]:
x_train.shape

(90570, 2625)

In [20]:
# feature_size = nbFeatures
# embedding_size = 100

# x = tf.placeholder(tf.float32, [None, feature_size] , name='features')
# y = tf.placeholder(tf.float32, [None] , name='labels')

# embeddings = tf.Variable(tf.random_normal([feature_size, embedding_size], mean=0.0, stddev=0.01), 
#                          name='feature_embeddings') # 
# feature_value = tf.reshape(tf.cast(x,tf.float32), shape=[-1, feature_size, 1])
# w0 = tf.Variable(tf.random_normal([1], mean=0.0, stddev=0.01))

# # ------ first-order term ------
# feature_bias = tf.Variable(tf.random_uniform([feature_size, 1], 0.0, 1.0), name='feature_bias') # 初始化w_i
# y_first_order = feature_bias
# y_first_order = tf.reduce_sum(tf.multiply(y_first_order, feature_value), axis=2) # w_i * x_i  (n * k)

# # ------ second-order term ------
# # 和平方部分
# embeddings = tf.multiply(embeddings, feature_value) # v_i * x_i
# summed_feature_emb = tf.reduce_sum(embeddings,axis=1)     # n * k
# summed_feature_emb_square = tf.square(summed_feature_emb) # n * k 

# # 平方和部分
# squared_feature_emb = tf.square(embeddings)
# squared_sum_feature_emb = tf.reduce_sum(squared_feature_emb, axis = 1)

# # 和平方部分与平方和部分组合
# y_second_order = 0.5 * tf.subtract(summed_feature_emb_square, squared_sum_feature_emb)

# y_predict = tf.add(tf.reduce_sum(y_first_order,1), tf.reduce_sum(y_second_order,1))
# y_predict = tf.add(w0, y_predict)
# # y_predict = tf.nn.sigmoid(y_predict)

# epochs = 200

# # loss定义为均方差
# loss = tf.reduce_mean(tf.square(y_predict - y))
# auc = tf.contrib.metrics.streaming_auc(y_predict,tf.convert_to_tensor(y))  
# train = tf.train.AdamOptimizer(learning_rate=0.01,epsilon=1e-8).minimize(loss)



# ## 
# with tf.Session() as sess:
#     init = tf.global_variables_initializer()
#     sess.run(init)
#     sess.run(tf.local_variables_initializer())
#     print('training...')
#     for _ in range(epochs):
#         sess.run(train, feed_dict = {x:x_train, y: y_train})
#         if _ % 10 == 0:
#             print("[%d]loss:%s"%(_,sess.run(loss, feed_dict={x: x_train, y: y_train})))
    
#     print('testing...')
#     print('MSE:',sess.run(loss, feed_dict={x: x_test ,y: y_test}))
#     print('AUC:',sess.run(auc, feed_dict={x: x_test ,y: y_test}))

In [24]:
## 稀疏矩阵操作
feature_size = nbFeatures
embedding_size = 100

x = tf.sparse_placeholder(tf.float32, [None, feature_size] , name='features')
y = tf.placeholder(tf.float32, [None] , name='labels')

embeddings = tf.Variable(tf.random_normal([feature_size, embedding_size], mean=0.0, stddev=0.01), 
                         name='feature_embeddings') # 
feature_value = tf.sparse_reshape(x, shape=[-1, feature_size, 1])
w0 = tf.Variable(tf.random_normal([1], mean=0.0, stddev=0.01))

# ------ first-order term ------
feature_bias = tf.Variable(tf.random_uniform([feature_size, 1], 0.0, 1.0), name='feature_bias') # 初始化w_i
y_first_order = tf.sparse_reduce_sum(tf.sparse_tensor_dense_matmul(feature_bias, tf.sparse_transpose(feature_value)), axis=2) # w_i * x_i  (n * k)

# ------ second-order term ------
# 和平方部分
embeddings = tf.multiply(embeddings, feature_value) # v_i * x_i
summed_feature_emb = tf.reduce_sum(embeddings,axis=1)     # n * k
summed_feature_emb_square = tf.square(summed_feature_emb) # n * k 

# 平方和部分
squared_feature_emb = tf.square(embeddings)
squared_sum_feature_emb = tf.reduce_sum(squared_feature_emb, axis = 1)

# 和平方部分与平方和部分组合
y_second_order = 0.5 * tf.subtract(summed_feature_emb_square, squared_sum_feature_emb)

y_predict = tf.add(tf.reduce_sum(y_first_order,1), tf.reduce_sum(y_second_order,1))
y_predict = tf.add(w0, y_predict)
# y_predict = tf.nn.sigmoid(y_predict)

epochs = 200

# loss定义为均方差
loss = tf.reduce_mean(tf.square(y_predict - y))
auc = tf.contrib.metrics.streaming_auc(y_predict,tf.convert_to_tensor(y))  
train = tf.train.AdamOptimizer(learning_rate=0.01,epsilon=1e-8).minimize(loss)

TypeError: Input must be a SparseTensor.

In [11]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    sess.run(tf.local_variables_initializer())
    print('training...')
    for _ in range(epochs):
        sess.run(train, feed_dict = {x:x_train, y: y_train})
        if _ % 10 == 0:
            print("[%d]loss:%s"%(_,sess.run(loss, feed_dict={x: x_train, y: y_train})))
    
    print('testing...')
    print('MSE:',sess.run(loss, feed_dict={x: x_test ,y: y_test}))
    print('AUC:',sess.run(auc, feed_dict={x: x_test ,y: y_test}))

training...


ResourceExhaustedError: OOM when allocating tensor with shape[90570,2625,100]
	 [[Node: Mul_1 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](feature_embeddings/read, Reshape)]]

Caused by op 'Mul_1', defined at:
  File "/home/zjw/anaconda3/envs/tf/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/zjw/anaconda3/envs/tf/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/zjw/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/zjw/anaconda3/envs/tf/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/zjw/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/zjw/anaconda3/envs/tf/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start
 

In [22]:
x_train.shape

(90570, 2625)

In [23]:
y_train.shape

(90570,)

In [44]:
x_train

<90570x2625 sparse matrix of type '<class 'numpy.float32'>'
	with 181140 stored elements in LInked List format>